In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from mixcoatl.database import Sensor, Segment, Result, db_session, query_results

database = './data/example_crosstalk_database.db'

# Exploring the Database

The crosstalk results database implementation uses `sqlalchemy` to interface with an `sqlite` database that contains three tables:
* `Sensor` table that holds information on the CCD sensors.
* `Segment` table that holds information on the segments of each of the CCD sensors.
* `Result` table that holds the measured crosstalk information for each segment.

The database can be interacted with directly using `sqlalchemy` [Object Relational Mapper or Core tools](https://docs.sqlalchemy.org/en/13/), or by using many of the `MixCOATL` built-in tools designed to handle common use cases.

## Basic Database Interfacing

Because of the capabilities for a database to become corrupted or unfinished changes to be pushed to the output file, it is highly recommended that all interactions with a database file be performed using the `db_session` context manager, which will safely handle session committing, rollback, and file closure. This tutorial will use this interface extensivel.

First, we'll demonstrate this functionality by connecting to a database and displaying a brief summary of the table data.

In [2]:
with db_session(database) as session:
    
    ## Query for all sensors
    query = session.query(Sensor)
    sensors = query.all()
    
    for sensor in sensors:
        print(sensor.id, sensor.sensor_name, sensor.lsst_num, sensor.manufacturer, sensor.namps)

1 Davis E2V-CCD250-112-09 E2V 16


The information associated with each sensor is a unique integer id, an informal name, the official designation, the manufacturer, and the number of amplifiers. The unique integer id is the "primary key" used to uniquely identify each sensor in the database. 

We can perform a similar query to display information regarding the segments included in the `Segment` table of the database.

In [3]:
with db_session(database) as session:
    
    ## Query for all segments
    query = session.query(Segment)
    segments = query.all()
    
    for segment in segments:
        print(segment.id, segment.segment_name, segment.amplifier_number, segment.sensor_id)

1 C17 1 1
2 C16 2 1
3 C15 3 1
4 C14 4 1
5 C13 5 1
6 C12 6 1
7 C11 7 1
8 C10 8 1
9 C00 9 1
10 C01 10 1
11 C02 11 1
12 C03 12 1
13 C04 13 1
14 C05 14 1
15 C06 15 1
16 C07 16 1


The information associated with each segment is a unique integer id (the primary key), the segment name, the output amplifier number, and the primary key integer id of the associated sensor.  

To explore the basic structure of the `Result` table, instead of returning all of the results with the query, we'll instead select only the results associated with a specific segment (using that segment's primary key), and print only a subset of those results.

In [4]:
with db_session(database) as session:
    
    ## Query results for specific segment
    query = session.query(Result).filter(Result.aggressor_id == 4)
    results = query.all()
    
    for result in results[:16]:
        print(result.id, result.aggressor_id, result.aggressor_signal, result.coefficient, result.methodology, result.victim_id)

9 4 280.0465087890625 0.00043603907663233805 MODEL_LSQ 1
10 4 280.0465087890625 0.0004445579917056866 MODEL_LSQ 2
11 4 280.0465087890625 0.0021998089675638725 MODEL_LSQ 3
12 4 280.0465087890625 1.0000000437061902 MODEL_LSQ 4
13 4 280.0465087890625 0.0003210460828854777 MODEL_LSQ 5
14 4 280.0465087890625 6.472202384360737e-05 MODEL_LSQ 6
15 4 280.0465087890625 0.00021731134204457762 MODEL_LSQ 7
16 4 280.0465087890625 0.0003629123249645695 MODEL_LSQ 8
41 4 276.6744384765625 0.0003746863855702538 MODEL_LSQ 1
42 4 276.6744384765625 0.0005173852160293196 MODEL_LSQ 2
43 4 276.6744384765625 0.0021559645142792848 MODEL_LSQ 3
44 4 276.6744384765625 1.0000000436049534 MODEL_LSQ 4
45 4 276.6744384765625 0.00016765589981633657 MODEL_LSQ 5
46 4 276.6744384765625 0.0001724513556352345 MODEL_LSQ 6
47 4 276.6744384765625 0.00029766953213234033 MODEL_LSQ 7
48 4 276.6744384765625 0.00014668737377684075 MODEL_LSQ 8


The information associated with each result is a unique integer id (the primary key), the integer id of the associated aggressor segment, the signal value of the aggressor, the crosstalk coefficient, the measurement method, and the integer id of the associated victim segment.

The above examples have demonstrated the basic means to query the database using the `sqlalchemy` ORM tools, by first building `Query` objects and appending `join` and `filter` methods to the query, before returning the results using `Query.one()`, `Query.first()`, or `Query.all()`.

## MixCOATL Database Tools

The ability to establish links between tables and corresponding table objects allows for easier querying capabilities to be used within `MixCOATL`.  This is demonstrated below by first using the classmethod `Sensor.from_db()` to initialize a `Sensor` object by querying the database.  The required query keyword is either `sensor_name` or `lsst_num`.

The returned `Sensor` object can access the associated `Segment` objects as a class attribute `Sensor.segments` that is a dictionary of the associated `Segment` objects, using the output amplifier numbers `Segment.amplifier_number` as the dictionary keys.

Similarly, the `Result` objects associated with each `Segment` in `Sensor.segments` can be accessed as a class attribute `Segment.results` that is a list of all the `Result` objects.

In [5]:
with db_session(database) as session:
    
    sensor = Sensor.from_db(session, sensor_name='Davis')
    
    for key in sensor.segments:
        segment_name = sensor.segments[key].segment_name
        results = sensor.segments[key].results
        
        print(key, segment_name, len(results))

1 C17 0
2 C16 0
3 C15 4256
4 C14 4256
5 C13 0
6 C12 0
7 C11 0
8 C10 0
9 C00 0
10 C01 0
11 C02 0
12 C03 0
13 C04 0
14 C05 4256
15 C06 4256
16 C07 0


In this manner, a user can easily query the database for a specific sensor, given knowledge of the sensor's informal name or official designation, and have access to all the crosstalk result associated with that sensor.

`MixCOATL` also offers tools to easily query the database for a specific segment, given knowledge of that segments "parent" sensor, demonstrated below. Here the classmethod `Segment.from_db()` is used to initialize a `Segment` object by querying the database.  The required keywords are one of either `segment_name` or `amplifier_number` and one of either `sensor_name` or `lsst_num`.

In [6]:
with db_session(database) as session:
    
    segment = Segment.from_db(session, amplifier_number=4, sensor_name='Davis')
    results = segment.results
    
    print(segment.id, segment.segment_name, len(results))

4 C14 4256


The final convenience tool that will be demonstrated is the capability to, given a specific sensor, query for all the crosstalk results for a given aggressor and victim amplifier pair. This is done using the convenience function `query_results`.

In [7]:
with db_session(database) as session:
    
    results = query_results(session, 'Davis', 4, 5, methods='MODEL_LSQ')
    print(len(results))

532
